<table>
    <tr><td>
         <a href="https://nbviewer.jupyter.org/github/panayiotiska/Jupyter-Sentiment-Analysis-Video-games-reviews/blob/master/[Data_Exploration]Word-Clouds.ipynb">
         <img alt="start" src="figures/button_previous.jpg" width= 70% height= 70%>
    </td><td>
        <a href="https://nbviewer.jupyter.org/github/panayiotiska/Jupyter-Sentiment-Analysis-Video-games-reviews/blob/master/Index.ipynb">
         <img alt="start" src="figures/button_table-of-contents.jpg" width= 70% height= 70%>
    </td><td>
         <a href="https://nbviewer.jupyter.org/github/panayiotiska/Jupyter-Sentiment-Analysis-Video-games-reviews/blob/master/Naive_Bayes_&_SVM_HashingVectorizer_Binary.ipynb">
         <img alt="start" src="figures/button_next.jpg" width= 70% height= 70%>
    </td></tr>
</table>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from collections import Counter

#[1] Importing dataset

dataset = pd.read_json(r"C:\Users\Panos\Desktop\Dissert\Code\Video_Games_5.json", lines=True, encoding='latin-1')
dataset = dataset[['reviewText','overall']]

#[2] Reduce number of classes

ratings = []
for index,entry in enumerate(dataset['overall']):
    if entry == 1.0 or entry == 2.0:
        ratings.append(-1)
    elif entry == 3.0:
        ratings.append(0)
    elif entry == 4.0 or entry == 5.0:
        ratings.append(1)

In [2]:
#[3] Cleaning the text

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['reviewText'][i])
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Panos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#[4] Prepare Train and Test Data sets
            
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(corpus,ratings,test_size=0.3)

print(Counter(Train_Y).values()) # counts the elements' frequency

dict_values([19749, 122618, 19879])


In [4]:
#[5] Encoding

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [5]:
#[6] Word Vectorization
        
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(corpus)
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# the vocabulary that it has learned from the corpus
#print(Tfidf_vect.vocabulary_)

# the vectorized data
#print(Train_X_Tfidf)

In [6]:
#[7] Use the Naive Bayes Algorithms to Predict the outcome

# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("-----------------------Naive Bayes------------------------\n")
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
# Making the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Test_Y, predictions_NB)
print("\n",cm,"\n")
# Printing a classification report of different metrics
from sklearn.metrics import classification_report
my_tags = ['Positive','Neutral','Negative']
print(classification_report(Test_Y, predictions_NB,target_names=my_tags))

# Export reports to files for later visualizations
report_NB = classification_report(Test_Y, predictions_NB,target_names=my_tags, output_dict=True)
report_NB_df = pd.DataFrame(report_NB).transpose()
report_NB_df.to_csv(r'NB_report_TFIDFVect.csv', index = True, float_format="%.3f")

-----------------------Naive Bayes------------------------

Naive Bayes Accuracy Score ->  77.46282394224409

 [[ 1544   120  6973]
 [  174   118  8234]
 [  121    49 52201]] 

              precision    recall  f1-score   support

    Positive       0.84      0.18      0.29      8637
     Neutral       0.41      0.01      0.03      8526
    Negative       0.77      1.00      0.87     52371

    accuracy                           0.77     69534
   macro avg       0.68      0.40      0.40     69534
weighted avg       0.74      0.77      0.70     69534



In [7]:
#[8] Use the Support Vector Machine Algorithms to Predict the outcome

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("-----------------Support Vector Machine CM------------------\n")
print("Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
cm = confusion_matrix(Test_Y, predictions_SVM)
# Making the confusion matrix
print("\n",cm,"\n")
# Printing a classification report of different metrics
print(classification_report(Test_Y, predictions_SVM,target_names=my_tags))

# Export reports to files for later visualizations
report_SVM = classification_report(Test_Y, predictions_SVM,target_names=my_tags, output_dict=True)
report_SVM_df = pd.DataFrame(report_SVM).transpose()
report_SVM_df.to_csv(r'SVM_report_TFIDFVect.csv', index = True, float_format="%.3f")

-----------------Support Vector Machine CM------------------

Accuracy Score ->  82.27485834268128

 [[ 4993   761  2883]
 [ 1365  1399  5762]
 [  880   674 50817]] 

              precision    recall  f1-score   support

    Positive       0.69      0.58      0.63      8637
     Neutral       0.49      0.16      0.25      8526
    Negative       0.85      0.97      0.91     52371

    accuracy                           0.82     69534
   macro avg       0.68      0.57      0.59     69534
weighted avg       0.79      0.82      0.79     69534



<a href="https://nbviewer.jupyter.org/github/panayiotiska/Jupyter-Sentiment-Analysis-Video-games-reviews/blob/master/Naive_Bayes_&_SVM_HashingVectorizer_Binary.ipynb">
         <img alt="start" src="figures/button_next.jpg">